In [11]:
!pip install openai

Looking in indexes: https://mirrors.aliyun.com/pypi/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 948.6/948.6 kB 963.3 kB/s  0:00:011m951.0 kB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 878.7 kB/s  0:00:02a 0:00:01m eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8/8 [openai]━━━━ 7/8 [openai]c]


In [14]:
from openai import OpenAI

client = OpenAI(api_key="sk-cd6890b8ac624127af85f7094d7c534d", base_url="https://api.deepseek.com")

In [15]:
# Round 1
messages = [{"role": "user", "content": "9.11 和 9.8，哪个更大？"}]
response = client.chat.completions.create(
    model="deepseek-reasoner",
    messages=messages
)

In [17]:
response

ChatCompletion(id='76b551ad-2ec5-40f6-9666-d49667bf725b', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='9.11 和 9.8 比较，9.8 更大。\n\n理由：\n- 9.8 可以表示为 9.80，而 9.11 是 9.11。\n- 比较小数点后的数字：80 大于 11，因此 9.80 > 9.11。\n- 所以，9.8 > 9.11。', refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=None, reasoning_content='首先，用户的问题是“9.11 和 9.8，哪个更大？”这看起来像是在比较两个数字：9.11 和 9.8。我需要确定哪个数字更大。\n\n9.11 和 9.8 都是小数。9.11 表示 9 又 11 百分之一，而 9.8 表示 9 又 8 十分之一。为了比较它们，我应该将它们转换为相同的位数。\n\n9.8 可以写成 9.80，这样我就可以比较 9.11 和 9.80。现在，比较小数点后的部分：11 和 80。显然，80 大于 11，所以 9.80 大于 9.11。因此，9.8 大于 9.11。\n\n我也可以考虑数值：9.11 是 9.11，而 9.8 是 9.80，所以 9.80 更大。\n\n所以，答案应该是 9.8 更大。\n\n用户是用中文问的，所以我应该用中文回复。'))], created=1759116561, model='deepseek-reasoner', object='chat.completion', service_tier=None, system_fingerprint='fp_8333852bec_prod0820_fp8_kvcache', usage=CompletionUsage(completion_tokens=308, prompt_tokens=17, total_tokens=325, completion_tokens_details

## 方法一：直接使用强制转换方法

In [20]:
response_to_list = list(response)
response_to_list
select1 = response_to_list[1][1]
print(select1)

[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='9.11 和 9.8 比较，9.8 更大。\n\n理由：\n- 9.8 可以表示为 9.80，而 9.11 是 9.11。\n- 比较小数点后的数字：80 大于 11，因此 9.80 > 9.11。\n- 所以，9.8 > 9.11。', refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=None, reasoning_content='首先，用户的问题是“9.11 和 9.8，哪个更大？”这看起来像是在比较两个数字：9.11 和 9.8。我需要确定哪个数字更大。\n\n9.11 和 9.8 都是小数。9.11 表示 9 又 11 百分之一，而 9.8 表示 9 又 8 十分之一。为了比较它们，我应该将它们转换为相同的位数。\n\n9.8 可以写成 9.80，这样我就可以比较 9.11 和 9.80。现在，比较小数点后的部分：11 和 80。显然，80 大于 11，所以 9.80 大于 9.11。因此，9.8 大于 9.11。\n\n我也可以考虑数值：9.11 是 9.11，而 9.8 是 9.80，所以 9.80 更大。\n\n所以，答案应该是 9.8 更大。\n\n用户是用中文问的，所以我应该用中文回复。'))]


In [36]:
messages = select1[0].message
content = messages.content
print(f"content: {content}")
print("-*-" * 50)
reasoning_content = messages.reasoning_content
print(f"reasoning_content: {reasoning_content}")

content: 9.11 和 9.8 比较，9.8 更大。

理由：
- 9.8 可以表示为 9.80，而 9.11 是 9.11。
- 比较小数点后的数字：80 大于 11，因此 9.80 > 9.11。
- 所以，9.8 > 9.11。
-*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*-
reasoning_content: 首先，用户的问题是“9.11 和 9.8，哪个更大？”这看起来像是在比较两个数字：9.11 和 9.8。我需要确定哪个数字更大。

9.11 和 9.8 都是小数。9.11 表示 9 又 11 百分之一，而 9.8 表示 9 又 8 十分之一。为了比较它们，我应该将它们转换为相同的位数。

9.8 可以写成 9.80，这样我就可以比较 9.11 和 9.80。现在，比较小数点后的部分：11 和 80。显然，80 大于 11，所以 9.80 大于 9.11。因此，9.8 大于 9.11。

我也可以考虑数值：9.11 是 9.11，而 9.8 是 9.80，所以 9.80 更大。

所以，答案应该是 9.8 更大。

用户是用中文问的，所以我应该用中文回复。


In [ ]:

# reasoning_content = response.choices[0].message.reasoning_content
# content = response.choices[0].message.content

In [ ]:
# Round 2
# messages.append({'role': 'assistant', 'content': content})
# messages.append({'role': 'user', 'content': "How many Rs are there in the word 'strawberry'?"})
# response = client.chat.completions.create(
#     model="deepseek-reasoner",
#     messages=messages
# )
# print(messages)

In [7]:
a = 1
print(a)

1


In [8]:
a = 100